# Notebook to compare different runs varying coupling constants
## Gather Plaquette, polyakov loop and Acceptance 
Author: Venkitesh Ayyar (vayyar@bu.edu) \
May 4, 2023

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sqlite3

import subprocess as sp
import os 
import glob
import pandas as pd
import itertools
import yaml
import sys

In [ ]:
import gvar as gv

In [ ]:
# %matplotlib widget

In [ ]:
%matplotlib notebook

In [ ]:
sys.path.append('modules')
from modules_parse_hmc_Grid import *
from modules_measurement import *


$$ \chi_\mathcal{O} = L^3 \left[ \langle \mathcal{O}^2 \rangle - {(\langle \mathcal{O} \rangle )}^2 \right] = L^3 \langle (\mathcal{O} - \bar{\mathcal{O}})^2\rangle $$ 

$$ \kappa_\mathcal{O} = \frac{\langle \mathcal{O}^4 \rangle - 4 \langle \mathcal{O}^3 \rangle \langle \mathcal{O} \rangle +6 \langle \mathcal{O}^2 \rangle {\langle \mathcal{O} \rangle} ^2 - 3 {\langle \mathcal{O} \rangle}^4 }{\chi_\mathcal{O}^2} = \frac{ \langle \left( \mathcal{O-\bar{\mathcal{O}}} \right) ^4 \rangle }{\chi_\mathcal{O}^2}$$ 

In [ ]:
def f_get_run_info_dict(dict1,input_dict,run_label):
    '''   
    Get dictionary with run info that will be added to Dataframe as columns
    '''
    keys=['Lx','Lt','beta','mf']
    for key in keys:
        dict1[key]=input_dict[key]
    
    run_key='beta-%s_mf-%s_Lx-%s_Lt-%s'%(input_dict['beta'],input_dict['mf'],input_dict['Lx'],input_dict['Lt'])
    
    if input_dict['F_action'] =='Mobius_dwf': ## Add Ls in label for DWF
        keys.append('dwf_Ls')
    
        
    run_key='beta-%s_mf-%s_Lx-%s_Lt-%s'%(input_dict['beta'],input_dict['mf'],input_dict['Lx'],input_dict['Lt'])
            
    if input_dict['F_action'] =='Mobius_dwf': ## Add Ls in label for DWF
        run_key+='_Ls-%s'%(input_dict['dwf_Ls'])

    dict1['run_label'] = run_label
    run_key+='_'+run_label 
    
    dict1['run_key']   = run_key
        
    return dict1

## Read from HMC table

In [8]:
### Read sql database
fname1='Stored_data/sql_dbase_files/parsed_hmc.db'

conn = sqlite3.connect(fname1)
df_read=pd.read_sql_query("SELECT * FROM test",conn)
df_read['Polyakov']=df_read['Polyakov'].astype(np.complex128)


In [9]:
df_read

,Plaquette,iter,Polyakov,Traj_time,Accept,Lx,Lt,beta,mf,run_label,run_key
0,0.217566,1.0,0.000441-0.004499j,101.589996,NaN,16,8,10.0,0.1,hot-start,beta-10.0_mf-0.1_Lx-16_Lt-8_Ls-16_hot-start
1,0.299022,2.0,0.005226-0.002714j,123.992996,NaN,16,8,10.0,0.1,hot-start,beta-10.0_mf-0.1_Lx-16_Lt-8_Ls-16_hot-start
2,0.339804,3.0,0.001690-0.001203j,132.701996,NaN,16,8,10.0,0.1,hot-start,beta-10.0_mf-0.1_Lx-16_Lt-8_Ls-16_hot-start
3,0.364053,4.0,0.002087+0.001814j,136.733994,NaN,16,8,10.0,0.1,hot-start,beta-10.0_mf-0.1_Lx-16_Lt-8_Ls-16_hot-start
4,0.378987,5.0,-0.000039+0.004351j,138.664001,NaN,16,8,10.0,0.1,hot-start,beta-10.0_mf-0.1_Lx-16_Lt-8_Ls-16_hot-start
...,...,...,...,...,...,...,...,...,...,...,...
8123,0.563848,2391.0,-0.000522-0.001461j,105.352997,1.0,16,8,10.8,0.1,cold-start,beta-10.8_mf-0.1_Lx-16_Lt-8_Ls-16_cold-start
8124,0.563748,2392.0,-0.005199-0.003383j,102.129997,1.0,16,8,10.8,0.1,cold-start,beta-10.8_mf-0.1_Lx-16_Lt-8_Ls-16_cold-start
8125,0.563748,2393.0,-0.005199-0.003383j,101.683998,0.0,16,8,10.8,0.1,cold-start,beta-10.8_mf-0.1_Lx-16_Lt-8_Ls-16_cold-start
8126,0.563525,2394.0,-0.008637-0.000436j,101.417999,1.0,16,8,10.8,0.1,cold-start,beta-10.8_mf-0.1_Lx-16_Lt-8_Ls-16_cold-start


## Read summary HMC data from table

In [ ]:
### Read sql database
fname2='Stored_data/sql_dbase_files/summary_hmc_data.db'
conn = sqlite3.connect(fname2)
df_summary=pd.read_sql_query("SELECT * FROM summary",conn)


# Convert strings to gvars for analysis
gvar_key_list=['plaq', 'sus_plaq', 'kurt_plaq', 'polyakov', 'sus_poly', 'kurt_poly', 'traj_time']
for key in gvar_key_list:    
    df_summary[key] = df_summary[key].apply(lambda x: gv.gvar(x))

In [ ]:
df_summary